In [ ]:
import os
import sys
sys.path.append("../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import copy
import torch
from datetime import datetime
from utils.helper import ModelConfig, color_print
from utils.dataset_utils.load_dataset import (
    load_data,
)
from utils.model_utils.load_model import load_model
from utils.model_utils.evaluate import evaluate_model, get_sparsity
from utils.dataset_utils.sampling import SamplingDataset
from utils.prune_utils.prune import (
    prune_magnitude,
    prune_concern_identification,
    recover_tangling_identification,
)

In [ ]:
name = "YahooAnswersTopics"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

checkpoint = None
model_config = ModelConfig(name, device)
num_labels = model_config.config["num_labels"]

model, tokenizer, checkpoint = load_model(model_config)

In [ ]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    name, batch_size=32, num_workers=48
)

In [ ]:
i = 0
color_print("Start Time:" + datetime.now().strftime("%H:%M:%S"))
color_print("#Module " + str(i) + " in progress....")
num_samples = 64

positive_samples = SamplingDataset(
    train_dataloader, i, num_samples, num_labels, True, 4, device=device
)
negative_samples = SamplingDataset(
    train_dataloader, i, num_samples, num_labels, False, 4, device=device
)
all_samples = SamplingDataset(
    train_dataloader, 200, 20, num_labels, False, 4, device=device
)

In [ ]:
print("Evaluate the original model")
result = evaluate_model(model, model_config, test_dataloader)

In [ ]:
module = copy.deepcopy(model)
prune_magnitude(
    module, include_layers=["attention", "intermediate", "output"], sparsity_ratio=0.1
)

In [ ]:
# result = evaluate_model(module, model_config, test_dataloader)
get_sparsity(module)

In [ ]:
prune_concern_identification(
    model,
    module,
    positive_samples,
    include_layers=["attention", "intermediate", "output"],
    sparsity_ratio=0.6,
)

In [ ]:
# result = evaluate_model(module, model_config, test_dataloader)
get_sparsity(module)

In [ ]:
recover_tangling_identification(
    model,
    module,
    negative_samples,
    recovery_ratio=0.1,
    include_layers=["attention", "intermediate", "output"],
)

In [ ]:
result = evaluate_model(module, model_config, test_dataloader)
# save_module(module, "Modules/", "citi_yahoo_40p.pt")
get_sparsity(module)